In [1]:
from efficient_apriori import apriori
import sqlalchemy as sql
import pandas as pd

In [2]:
import json

with open('mysql.json', encoding='utf-8') as con_json:
    con_dict = json.load(con_json)

user = con_dict['user']
passwd = con_dict['passwd']
host = con_dict['host']
database = con_dict['database']

In [11]:
# 数据加载
engine = sql.create_engine(
    'mysql+pymysql://{}:{}@{}/{}'.format(user, passwd, host, database))
query = 'SELECT * FROM bread_basket'
data = pd.read_sql_query(query, engine)

In [13]:
data['Item'] = data.Item.str.lower()

In [16]:
data = data.drop(data[data.Item == 'none'].index)

In [44]:
transactions = list(data.groupby('Transaction').agg(lambda x: set(x.Item.values))['Item'])

In [45]:
# 挖掘频繁项集和频繁规则
itemsets, rules = apriori(transactions, min_support=0.02,  min_confidence=0.5)
print('频繁项集：', itemsets)
print('关联规则：', rules)

频繁项集： {1: {('alfajores',): 344, ('bread',): 3096, ('brownie',): 379, ('cake',): 983, ('coffee',): 4528, ('cookies',): 515, ('farm house',): 371, ('hot chocolate',): 552, ('juice',): 365, ('medialuna',): 585, ('muffin',): 364, ('pastry',): 815, ('sandwich',): 680, ('scandinavian',): 275, ('scone',): 327, ('soup',): 326, ('tea',): 1350, ('toast',): 318, ('truffles',): 192}, 2: {('bread', 'cake'): 221, ('bread', 'coffee'): 852, ('bread', 'pastry'): 276, ('bread', 'tea'): 266, ('cake', 'coffee'): 518, ('cake', 'tea'): 225, ('coffee', 'cookies'): 267, ('coffee', 'hot chocolate'): 280, ('coffee', 'juice'): 195, ('coffee', 'medialuna'): 333, ('coffee', 'pastry'): 450, ('coffee', 'sandwich'): 362, ('coffee', 'tea'): 472, ('coffee', 'toast'): 224}}
关联规则： [{cake} -> {coffee}, {cookies} -> {coffee}, {hot chocolate} -> {coffee}, {juice} -> {coffee}, {medialuna} -> {coffee}, {pastry} -> {coffee}, {sandwich} -> {coffee}, {toast} -> {coffee}]
